In [1]:
_ = '''
transaction = contract.newEvent({
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1000000,
    'betsCloseTime': int(time.time()) + 3600,
    'measurePeriod': 3600,
    'liquidityPercent': 10000,
}).as_transaction()

transaction.json_payload()#.hash()
'''

### Juster-Maker:
Tool allowing to run events in cycle

In [2]:
from pytezos import pytezos
import time
from pprint import pprint
from utility import repeat_until_succeed
from bulk_sender import BulkSender
from event_emitter import EventCreationEmitter


# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [3]:
_ = '''
CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}
'''

In [4]:
address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
client = pytezos.using(
    key='../2021-05-27-tests-modelling/test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json'
)
contract = client.contract(address)

In [5]:
import time
time.time()

1626716892.1503263

In [6]:
from utility import date_to_timestamp

### TODO:
- starts_from parameter for events creators so it would be possible to bind events to the time slots

In [8]:
from asyncio import Queue

# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 50

operations_queue = Queue(MAX_QUEUE_SIZE)

events_params = [
    {
        'currency_pair': currency_pair,
        'first_at': int(date_to_timestamp('2021-07-19 20:45:00')),
        'target_dynamics': 1_000_000,
        'bets_period': period,
        'measure_period': period,
        'liquidity_percent': 10_000,
        'expiration_fee': 100_000,
        'measure_start_fee': 100_000
    } for currency_pair in ['XTZ-USD', 'BTC-USD']
    for period in [600, 900]
]

In [9]:
# for each event_params EventCreationEmitter is created:
emitters = [
    EventCreationEmitter(
        period=params['bets_period'],
        contract=contract,
        operations_queue=operations_queue,
        event_params=params)
    for params in events_params
]

emitters.append(
    BulkSender(period=60, client=client, operations_queue=operations_queue)
)

In [10]:
for emitter in emitters:
    emitter.run()

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626717300, 'measurePeriod': 600, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626717600, 'measurePeriod': 900, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626717300, 'measurePeriod': 600, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626717600, 'measurePeriod': 900, 'liquidityPercent': 10000}
making bulk of 4 operations
signed, result: {'chain_id': 'NetXxkAx4woPLyu', 'hash': 'opaqJGKwV2AuudPDcBbeeghikWkjmvDKTsVhKSsFXn7avwwvwJY', 'protocol': 'PsFLorenaUUuikDWvMDr6fGBRG8kt3e3D3fHoXK1j1BFRxeSH4i', 'branch': 'BMeJBr65FDLkSGsy3CFDQhwvhmiAxjrKVBgTE3aoXqvqt6Lyg4W', 'contents': [

### TODO: maybe make next_event func/iterator that returns time and params for the next events

### TODO: need to have some tests for LoopExecutor:
- for example that it runned function K times in given time

### TODO: need to test all this code before running in florencenet

In [11]:
for emitter in emitters:
    emitter.stop()